In [2]:
from avalanche.benchmarks import nc_benchmark
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics,\
    loss_metrics, timing_metrics, StreamConfusionMatrix, class_accuracy_metrics, bwt_metrics
from avalanche.logging import InteractiveLogger, TextLogger, WandBLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training import ICaRL
from avalanche.benchmarks.utils import AvalancheDataset
import torch
from torch.optim.adamw import AdamW
from torch.utils.data import TensorDataset
import numpy as np
from icarlmodel import *
from torchsummary import summary
from sklearn.preprocessing import StandardScaler
from utils import log_single_metric_no_viz

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

train = np.load('CICIDS2017/train.npy')
test = np.load('CICIDS2017/test.npy')
print("Finished loading data")

# Standardize features (fit on train only)
scaler = StandardScaler().fit(train[:, :-1])
train_x = scaler.transform(train[:, :-1])
test_x  = scaler.transform(test[:, :-1])
train_y = train[:, -1].astype(np.int64)
test_y  = test[:, -1].astype(np.int64)

# Create TensorDataset from standardized arrays
train_dataset = TensorDataset(torch.from_numpy(train_x).float(), torch.from_numpy(train_y))
test_dataset  = TensorDataset(torch.from_numpy(test_x).float(),  torch.from_numpy(test_y))




/home/afifhaziq/CLiteNet/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
Finished loading data


In [ ]:
# Pass datasets directly to nc_benchmark (no AvalancheDataset wrapper needed)
benchmark = nc_benchmark(
         train_dataset, test_dataset,
         n_experiences=4,
         task_labels=False,
         shuffle=True, seed=1234,
         class_ids_from_zero_in_each_exp=False,   
         fixed_class_order=[0,1,2,3,4,5,6,7]      
     )


# Get input size from the dataset
input_size = train_dataset[0][0].shape[0]  # First sample's feature dimension
hidden_size = 64


feature_extractor = LiteNet(hidden_size)
classifier = LiteNetHead(hidden_size, benchmark.n_classes)
#feature_extractor = FeatureExtractor(input_size, hidden_size)
#classifier = ClassifierHead(hidden_size, benchmark.n_classes)
#print(benchmark.n_classes)

#print(summary(feature_extractor))
#summary(classifier, (64, 78))

WandBLogger.log_single_metric = log_single_metric_no_viz
logger = [InteractiveLogger()]
logger.append(TextLogger(open('ICaRL_log.txt', 'w')))
logger.append(WandBLogger(project_name="CL-ICaRL", 
                          params={"mode": "online"}))

eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    class_accuracy_metrics(experience=True),
    loss_metrics(epoch_running=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    timing_metrics(epoch=True, experience=True),
    StreamConfusionMatrix(num_classes=benchmark.n_classes, save_image=False),
    loggers=logger
)

criterion = torch.nn.CrossEntropyLoss()
optimizer = AdamW(list(feature_extractor.parameters()) + list(classifier.parameters()), lr=0.001, weight_decay=0.0001)

# CREATE THE STRATEGY INSTANCE
cl_strategy = ICaRL(
    feature_extractor=feature_extractor,
    classifier=classifier,
    optimizer=optimizer,
    memory_size=500,
    buffer_transform=None,
    fixed_memory=False,
    train_mb_size=128,
    train_epochs=10,
    eval_mb_size=256,
    device=device,
    evaluator=eval_plugin)


In [ ]:
# TRAINING LOOP
print('Starting experiment...')
results = []

for experience in benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience, num_workers=8)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # eval also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(benchmark.test_stream, num_workers=16))

import wandb
wandb.finish()

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1]
-- >> Start of training phase << --
100%|██████████| 4137/4137 [00:21<00:00, 195.25it/s]
Epoch 0 ended.
	RunningLoss_Epoch/train_phase/train_stream/Task000 = 0.0070
	Time_Epoch/train_phase/train_stream/Task000 = 21.1698
100%|██████████| 4137/4137 [00:24<00:00, 170.87it/s]
Epoch 1 ended.
	RunningLoss_Epoch/train_phase/train_stream/Task000 = 0.0017
	Time_Epoch/train_phase/train_stream/Task000 = 21.1489
100%|██████████| 4137/4137 [00:24<00:00, 168.73it/s]
Epoch 2 ended.
	RunningLoss_Epoch/train_phase/train_stream/Task000 = 0.0011
	Time_Epoch/train_phase/train_stream/Task000 = 21.4252
100%|██████████| 4137/4137 [00:21<00:00, 193.67it/s]
Epoch 3 ended.
	RunningLoss_Epoch/train_phase/train_stream/Task000 = 0.0008
	Time_Epoch/train_phase/train_stream/Task000 = 21.3410
100%|██████████| 4137/4137 [00:24<00:00, 169.13it/s]
Epoch 4 ended.
	RunningLoss_Epoch/train_phase/train_stream/Task000 = 0.0007
	Time_Epoch/train_phase/tra

/pytorch/aten/src/ATen/native/cuda/Loss.cu:90: operator(): block: [7,0,0], thread: [26,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:90: operator(): block: [3,0,0], thread: [26,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:90: operator(): block: [2,0,0], thread: [26,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:90: operator(): block: [1,0,0], thread: [26,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:90: operator(): block: [0,0,0], thread: [26,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:90: operator(): block: [4,0,0], thread: [26,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:90: operator(): block: [5,0,0], thread: [26,0,0] Assertio

AcceleratorError: CUDA error: device-side assert triggered
Search for `cudaErrorAssert' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


  5%|▌         | 35/690 [00:18<00:45, 14.49it/s]